In [1]:
import os
import xarray as xr
import glob
from datetime import date
import functools
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import pandas as pd

In [2]:
# Setup PBSCluster
cluster = PBSCluster(
    cores=1,                                      # The number of cores you want
    memory='10GB',                                # Amount of memory
    processes=1,                                  # How many processes
    queue='casper',                               # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/work/afoster',        # Use your local directory
    resource_spec='select=1:ncpus=1:mem=10GB',    # Specify resources
    project='P93300041',                          # Input your project ID here
    walltime='02:00:00',                          # Amount of wall time
    interface='ext',                              # Interface to use
)

/glade/work/afoster/conda-envs/ml_analysis/lib/python3.11/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/work/afoster/conda-envs/ml_analysis/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42357 instead
  warnings.warn(


In [3]:
cluster.scale(2)

/glade/work/afoster/conda-envs/ml_analysis/lib/python3.11/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


In [4]:
client = Client(cluster)

In [5]:
def preprocess(ds, varset):
    return ds[data_vars]

In [6]:
def postprocess(top_dir, histdir, data_vars):

    files = sorted(glob.glob(os.path.join(top_dir, histdir, 'lnd', 'hist/') +
                             "*clm2.h0*.nc"))
    flen = len(files)

    if flen < 12:

        return None

    else:

        ds = xr.open_mfdataset(files, combine='nested', concat_dim='time',
                               preprocess=functools.partial(preprocess,
                                                            varset=data_vars),
                               parallel=True, autoclose=True)

        # fix time bug
        ds['time'] = xr.cftime_range(str(2005), periods=12*60, freq='MS')
        ds = ds.sel(time=slice("2055-01-01", "2064-12-31"))
        ds['time'] = xr.cftime_range(str(2005), periods=12*10, freq='MS')

        # calculate some variables
        ds['ASA'] = ds.FSR/ds.FSDS.where(ds.FSDS > 0)
        ds['ASA'].attrs['units'] = 'unitless'
        ds['ASA'].attrs['long_name'] = 'All sky albedo'

        ds['GPP'] = ds['FATES_GPP']*ds['FATES_FRACTION']  # kg m-2 s-1
        ds['GPP'].attrs['units'] = ds['FATES_GPP'].attrs['units']
        ds['GPP'].attrs['long_name'] = ds['FATES_GPP'].attrs['long_name']

        ds['Temp'] = ds.TSA - 273.15
        ds['Temp'].attrs['units'] = 'degrees C'
        ds['Temp'].attrs['long_name'] = ds['TSA'].attrs['long_name']

        ds0 = xr.open_dataset(files[0])
        extras = ['grid1d_lat', 'grid1d_lon']
        for extra in extras:
            ds[extra] = ds0[extra]

        ds['ensemble'] = histdir.split('_')[-1]

        ds.attrs['Date'] = str(date.today())
        ds.attrs['Author'] = 'afoster@ucar.edu'
        ds.attrs['Original'] = files[0]

        return ds

In [8]:
data_vars = ['FATES_FRACTION', 'FATES_GPP', 'FATES_LAI', 'QVEGE', 'QVEGT',
             'QSOIL', 'EFLX_LH_TOT', 'FSH', 'QRUNOFF', 'SOILWATER_10CM',
             'FSR', 'FSDS', 'FSA', 'FIRE', 'FLDS', 'FGR12', 'RAIN',
             'SNOW', 'H2OSNO', 'SNOWDP', 'TSOI', 'TSA', 'TREFMXAV',
             'TREFMNAV', 'RH2M']

In [9]:
top_dir = '/glade/derecho/scratch/afoster/FATES_SP_LH_500_updated/archive'
postp_dir = '/glade/work/afoster/FATES_calibration/FATES_SP_LH/hist_500_updated/'
dirs = sorted(os.listdir(top_dir))

In [10]:
bad_ensembles = []
good_ensembles = []
for histdir in dirs:
    ensemble = histdir.split('_')[-1]
    ds = postprocess(top_dir, histdir, data_vars)
    if ds is not None:
        out_file = os.path.join(postp_dir, f"{histdir}.nc")
        ds.to_netcdf(out_file)
        good_ensembles.append(ensemble)
    else:
        bad_ensembles.append(ensemble)

In [ ]:
with open('ensembles_good.txt', 'w') as f:
    for line in good_ensembles:
        f.write(f"{line}\n")

In [12]:
bad_ensembles

['416']